In [9]:
# 필요한 라이브러리 임포트
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split

# MLRSNet 데이터셋의 루트 디렉토리 설정
mlrsnet_root = '/home/son/ml/datasets/MLRSNet'  # MLRSNet 데이터셋의 최상위 폴더 경로로 변경하세요

# 레이블 디렉토리 경로 설정
labels_dir = os.path.join(mlrsnet_root, 'Labels')

# 모든 레이블 CSV 파일 불러오기
label_files = glob.glob(os.path.join(labels_dir, '*.csv'))

# 모든 레이블 데이터를 하나의 데이터프레임으로 합치기
all_labels_df = pd.DataFrame()

for file in label_files:
    df = pd.read_csv(file)
    # 이미지 경로 수정: 이미지 파일명이 있는 폴더명을 추출하여 경로 생성
    class_name = os.path.splitext(os.path.basename(file))[0]  # 파일명에서 클래스명 추출
    # 이미지 경로 생성
    df['image'] = df['image'].apply(lambda x: os.path.join('Images', class_name, x))
    all_labels_df = pd.concat([all_labels_df, df], ignore_index=True)

# 이미지 경로에 MLRSNet 루트 경로를 추가
all_labels_df['image'] = all_labels_df['image'].apply(lambda x: os.path.join(mlrsnet_root, x))

# 데이터 섞기
all_labels_df = all_labels_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 데이터 분할 비율 설정 (예: 70% train, 15% val, 15% test)
train_ratio = 0.4
val_ratio = 0.1
test_ratio = 0.5

# 먼저 train과 나머지(val+test)를 분할
train_df, temp_df = train_test_split(all_labels_df, test_size=(1 - train_ratio), random_state=42)

# 남은 데이터(temp_df)를 val과 test로 분할
val_df, test_df = train_test_split(temp_df, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=42)

# 각 분할의 데이터 수 확인
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

# CSV 파일 저장 경로 설정
output_dir = '/home/son/ml/xai_new/split_data_v2'  # 현재 작업 디렉토리로 설정하거나 원하는 경로로 변경하세요

# CSV 파일로 저장
train_df.to_csv(os.path.join(output_dir, 'train.csv'), index=False)
val_df.to_csv(os.path.join(output_dir, 'val.csv'), index=False)
test_df.to_csv(os.path.join(output_dir, 'test.csv'), index=False)

Train set size: 43664
Validation set size: 10916
Test set size: 54581
